## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database-Copy1.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,288.15,55,40,15.43,scattered clouds
1,1,Georgetown,MY,5.4112,100.3354,301.15,78,20,1.03,few clouds
2,2,Jamestown,US,42.0970,-79.2353,271.48,86,90,3.60,light snow
3,3,Mirnyy,RU,62.5353,113.9611,235.81,82,100,2.93,overcast clouds
4,4,Qaanaaq,GL,77.4840,-69.3632,249.31,80,78,2.45,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 300
What is the maximum temperature you would like for your trip? 350


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Georgetown,MY,5.4112,100.3354,301.15,78,20,1.03,few clouds
16,16,Kirakira,SB,-10.4544,161.9205,300.67,81,100,11.15,overcast clouds
22,22,Puerto Ayora,EC,-0.7393,-90.3518,303.15,66,0,5.14,clear sky
24,24,Arraial Do Cabo,BR,-22.9661,-42.0278,306.15,51,0,6.17,clear sky
29,29,Mahebourg,MU,-20.4081,57.7000,300.15,89,75,3.60,light rain
...,...,...,...,...,...,...,...,...,...,...
674,674,Maragogi,BR,-9.0122,-35.2225,301.41,67,38,6.48,scattered clouds
681,681,Durazno,UY,-33.4131,-56.5006,300.15,65,75,4.22,broken clouds
683,683,Alice Springs,AU,-23.7000,133.8833,301.15,39,90,3.60,overcast clouds
688,688,Sokolo,ML,14.7328,-6.1219,302.23,12,97,4.51,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                138
City                   138
Country                137
Lat                    138
Lng                    138
Max Temp               138
Humidity               138
Cloudiness             138
Wind Speed             138
Current Description    138
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                137
City                   137
Country                137
Lat                    137
Lng                    137
Max Temp               137
Humidity               137
Cloudiness             137
Wind Speed             137
Current Description    137
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,301.15,few clouds,5.4112,100.3354,
16,Kirakira,SB,300.67,overcast clouds,-10.4544,161.9205,
22,Puerto Ayora,EC,303.15,clear sky,-0.7393,-90.3518,
24,Arraial Do Cabo,BR,306.15,clear sky,-22.9661,-42.0278,
29,Mahebourg,MU,300.15,light rain,-20.4081,57.7000,
32,Saldanha,ZA,300.15,clear sky,-33.0117,17.9442,
36,Geraldton,AU,304.15,broken clouds,-28.7667,114.6000,
45,Faanui,PF,300.30,overcast clouds,-16.4833,-151.7500,
50,Carnarvon,AU,303.15,overcast clouds,-24.8667,113.6333,
55,Telimele,GN,310.86,clear sky,10.9167,-13.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   137
Country                137
Max Temp               137
Current Description    137
Lat                    137
Lng                    137
Hotel Name             137
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, 
#the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °K</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template 
#and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))